In [51]:
from tqdm import tqdm
#from .autonotebook import tqdm as notebook_tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from statistics import mean
import math
torch.manual_seed(42)

# from sklearn import preprocessing

In [52]:
##dataset class
from torch.utils.data.dataset import Dataset, random_split

class ExpertDataSet(Dataset):

    def __init__(self, expert_observations, expert_actions):
        self.observations = torch.from_numpy(expert_observations)
        self.actions = self.preprocess_data(torch.from_numpy(expert_actions))

    def __getitem__(self, idx):
        # return (self.observations[index], self.actions[index])
        normalized_observations = 2 * ((self.observations[idx] - self.observations.min()) / (self.observations.max() - self.observations.min())) - 1
        # normalized_actions = 2 * ((self.actions[idx] - self.actions.min()) / (self.actions.max() - self.actions.min())) - 1
        normalized_data = (normalized_observations, self.actions[idx])
        return normalized_data


    def __len__(self):
        return len(self.observations)
    
    def preprocess_data(self, data, clip_value=1e38):
        # Clip values to a maximum and minimum range
        data = torch.clamp(data, min=-clip_value, max=clip_value)
        
        # Convert to float
        return data.float()

In [53]:
test_ = np.array([1,2,3,])

In [109]:
expert_observations = np.load('./expert-observations.npy', allow_pickle=True)
expert_actions = np.load('./expert-actions.npy', allow_pickle=True)

count_discarded_numpy = 0
count_discarded = 0


new_exp_action = expert_actions

list_of_index_to_drop = []
for i, a in enumerate(expert_actions):
  if (a > 1e2).any() or (a > 1e2).any():
  # if not np.isfinite(a).all(): 
    list_of_index_to_drop.append(i)
    print(i)
    print(a)
    count_discarded_numpy+=1
    # break


print(len(new_exp_action))
print(len(expert_observations))

expert_dataset = ExpertDataSet(expert_observations, expert_actions)



for i in range(len(expert_dataset)):
  a = expert_dataset.__getitem__(i)[1]
  print(a.max())
  print(a.min())
  if (a > 1e2).any() or (a < -1e2).any() :
  # if not torch.isfinite(a).any():
    count_discarded += 1
    print(a)

# print(expert_dataset.__getitem__(90)[1])

#split in 80% training and 20%test
batch_size = 64
train_prop = 0.8
train_size = int(train_prop * len(expert_dataset))
test_size = len(expert_dataset) - train_size
train_expert_dataset, test_expert_dataset = random_split(expert_dataset, [train_size, test_size])

train_loader = torch.utils.data.DataLoader(  dataset=train_expert_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(  dataset=test_expert_dataset, batch_size=batch_size, shuffle=True)


print(train_loader.dataset.__getitem__(0)[0].shape[0])
print(train_loader.dataset.__getitem__(0)[1].shape[0])



8000
8000
tensor(1.8602)
tensor(-1.3725)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(1.3818)
tensor(-1.0001)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(1.0914)
tensor(-0.3608)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(1.3145)
tensor(-0.5403)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(1.6139)
tensor(-0.6281)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)

In [110]:
print(count_discarded)
print(count_discarded_numpy)



0
0


In [117]:
###### Define student agent

no_cuda = False
torch_available = torch.cuda.is_available()
print(torch_available)
use_cuda = not no_cuda and torch.cuda.is_available()
print('use_cuda: ', use_cuda)

device = torch.device("cuda" if use_cuda else "cpu")

class StudentAgent:
    def __init__(self, train_loader, test_loader, learning_rate, threshold):
        self.train_loader = train_loader
        self.test_loader = test_loader

        n_inputs = train_loader.dataset.__getitem__(0)[0].shape[0] # dimension of observation space (of state = 196)
        n_outputs = train_loader.dataset.__getitem__(0)[1].shape[0] # dimension of actions space (of outptu = 36)

        #simple layer
        self.policy = nn.Sequential(
            # nn.BatchNorm1d(n_inputs),
            nn.Linear(n_inputs, 16), 
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Linear(16, n_outputs),
            # nn.BatchNorm1d(n_outputs)
            # nn.Softmax(dim=-1)
            )

        print("policy net: ", self.policy)

        # self.loss_criterion = nn.HuberLoss() #nn.MSELoss()
        self.loss_criterion = nn.MSELoss()

        self.optimizer =  optim.Adam(self.policy.parameters(), lr=learning_rate)
        self.num_eval_episodes = 10
        self.accuracy_threshold = threshold

    def print_gradients(self):
        for name, param in self.policy.named_parameters():
            if param.requires_grad:
                if torch.isnan(param.grad).any(): 
                    return 1#break
                # print(f"Gradient of {name}: {param.grad}")
        return 0

    def train(self, num_epochs):
        self.policy.train()
        self.policy.to(device)
        
        loss = 0
        epoch_loss = 0
        unused_val = 0

        for epoch in range(num_epochs):
            for batch_idx, (data, target) in enumerate(self.train_loader):
                obs, expert_action = data.to(device), target.to(device)

                self.optimizer.zero_grad()
                obs = obs.float()

                student_action = self.policy(obs)
                expert_action = expert_action.float()
                
                # print("### BATCH {} ###".format(batch_idx))
                # print(f'obs -> {obs}')


                loss = self.loss_criterion(student_action, expert_action)
                # loss.register_hook(lambda grad: print(grad))
                loss.backward()
                print("Loss: {}".format(loss.item()))
                
                
                if not loss.item() == torch.inf: 
                    epoch_loss += loss.item()
                    self.optimizer.step()

                else:
                    unused_val += 1
                    return expert_action,student_action
                    print("\n______________________________________________________________________________")
                    print(f'expert_action -> {expert_action}')
                    print("\n______________________________________________________________________________")
                    print(f'student_action -> {student_action}')
                    print("\n______________________________________________________________________________")

                res = self.print_gradients()
                
                # print("###############################################################################\n")

                if torch.isnan(student_action).any(): 
                    print('e successo')
                    break
                if res == 1: 
                    print("\n______________________________________________________________________________")
                    print(student_action.shape)
                    for i, ea in enumerate(expert_action):
                        if not np.isfinite(ea).all():
                            print(i+64)
                            print(f'expert_action -> {ea}')

                    print("\n______________________________________________________________________________")
                    print(f'Max expert_action -> {expert_action.max()}')
                    print(f'Min expert_action -> {expert_action.min()}')
                    print(f'Max student_action -> {student_action.max()}')
                    print(f'Min student_action -> {student_action.min()}')
                    break
                
                # print("Student actions: {}".format(student_action.shape))
                # print("Expert actions: {}".format(expert_action.shape))
            
            
            # compute accuracy
            train_acc = self.compute_accuracy(self.train_loader)
            test_acc = self.compute_accuracy(self.test_loader)
            print("Epoch {}:\ttrain accuracy: {}\ttest accuracy: {}".format(epoch, train_acc, test_acc))
            print("Epoch Loss: {}".format(epoch_loss/(batch_idx+1)))
            print("Unused loss: {}".format(unused_val))
            epoch_loss = 0
            unused_val = 0
            if train_acc >80. and test_acc>80.: return
            print("###############################################################################\n")


    def compute_accuracy(self, loader):
        total = 0
        correct = 0
        self.policy.eval()
        test_loss = 0

        with torch.no_grad():
            for data, target in loader:
                obs, expert_action = data.to(device), target.to(device)
                obs = obs.float()
                student_action = self.policy(obs)

                total += student_action.size(0)

                # print("Total: {}".format(total))
                # print("\n______________________________________________________________________________")
                # print(f'expert_action -> {expert_action}')
                # print("\n______________________________________________________________________________")
                # print(f'student_action -> {student_action}')
                # print("\n______________________________________________________________________________")
                # similarity = math.isclose(student_action.any(), expert_action.any(), rel_tol=1e-5)

                # difference = (expert_action - student_action).abs()
                similarity= torch.abs(student_action - expert_action)
                similarity = torch.sum(similarity,dim=1)/36
                similarity_torch= torch.mean(torch.abs(torch.square(student_action) - torch.square(expert_action)),dim=1)

                correct  += (similarity < self.accuracy_threshold).sum().item()
                # print(f'similarity -> {similarity.shape}') # 64
                print(f'similarity -> {similarity.mean()}')
                print(f'similarity -> {similarity.shape}')

                
                print(f'similarity torch -> {similarity_torch.mean()}')
                print(f'similarity -> {similarity_torch.shape}')


                # print(f'correct -> {correct}')
                # correct += int(similarity/total)
                # correct += sum(student_action==expert_action).item()
                

        print(f'correct -> {correct}')
        accuracy = 100. * correct/(total)

        return accuracy

student = StudentAgent(train_loader, test_loader, learning_rate=1e-2, threshold=1e-3)

False
use_cuda:  False
policy net:  Sequential(
  (0): Linear(in_features=196, out_features=16, bias=True)
  (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Linear(in_features=16, out_features=36, bias=True)
)


In [118]:
prova = student.train(10)

Loss: 0.19927382469177246
Loss: 0.10401061177253723
Loss: 0.12193446606397629
Loss: 0.08147257566452026
Loss: 0.09513695538043976
Loss: 0.06662895530462265
Loss: 0.08355125784873962
Loss: 0.0636349618434906
Loss: 0.04961477220058441
Loss: 0.045096542686223984
Loss: 0.08336959034204483
Loss: 0.07349104434251785
Loss: 0.028915338218212128
Loss: 0.06111045181751251
Loss: 0.038469526916742325
Loss: 0.049060311168432236
Loss: 0.035233523696660995
Loss: 0.027821604162454605
Loss: 0.03372935578227043
Loss: 0.01794547401368618
Loss: 0.02942008525133133
Loss: 0.031083719804883003
Loss: 0.01665334217250347
Loss: 0.017926372587680817
Loss: 0.0445534773170948
Loss: 0.0288105309009552
Loss: 0.05873262882232666
Loss: 0.009935223497450352
Loss: 0.03232155367732048
Loss: 0.034921903163194656
Loss: 0.02042781002819538
Loss: 0.030025340616703033
Loss: 0.04165019094944
Loss: 0.019012557342648506
Loss: 0.036315202713012695
Loss: 0.02470231056213379
Loss: 0.024590283632278442
Loss: 0.04515599459409714
Loss

In [ ]:
prova2= studen

In [58]:

n1,n2 = prova
print(n1.max())
print(n2.max())
print(torch.mean(n1-n2))


TypeError: cannot unpack non-iterable NoneType object